In [ ]:
import boto3

def analyze_regulatory_impact(company, kb_dict):
    """
    Analyze the regulatory impact on a company across multiple knowledge bases.
    
    Args:
        company (str): Name of the company to analyze
        kb_dict (dict): Dictionary with KB IDs as keys and document names as values
    
    Returns:
        list: List of dictionaries containing analysis results for each KB
    """
    # Define the predefined prompts
    predefined_prompt = f"""Based on the regulations and laws described in the knowledge base, predict the effect these regulations will have on a company like {company}. Note that many regulations impose compliance costs and operational challenges on companies, often leading to negative impacts on stock prices and business operations, though some may present opportunities for competitive advantages or market expansion. Analyze whether the impact will be positive or negative, considering factors such as compliance costs, market opportunities, competitive advantages, and overall business implications.

Provide a detailed analysis and prediction in the following JSON format:
{{
"reasoning": "Detailed explanation based on the regulations",
"impact_on_business": "Description of how it affects operations",
"stock_price_effect": number from -10 (very negative) to +10 (very positive),
"importance_level": number from 0 (minimal impact) to 10 (highly significant)
}}"""
    
    generation_prompt = """You are an expert financial and regulatory analyst. You have access to search results from a knowledge base containing information about regulations and laws.

Your task is to analyze the provided search results and predict the impact of the described regulations on a specific company. Focus on whether the regulations will have a positive, negative, or neutral effect, and provide quantitative assessments where possible.

Search Results:
$search_results$

Based on these results and your knowledge, provide a prediction in the exact JSON format below. Do not include any other text.

{{
"reasoning": "Comprehensive analysis explaining the prediction",
"impact_on_business": "Specific effects on company operations and strategy",
"stock_price_effect": number from -10 (very negative) to +10 (very positive),
"importance_level": number from 0 (minimal impact) to 10 (highly significant)
}}"""
    
    orchestration_prompt = """You are a query optimization agent for regulatory analysis. Your role is to generate the most effective search queries to retrieve relevant information about regulations and their potential impacts on companies.

Given the user's question about predicting regulatory effects on a company, create optimal queries that will find:
- Specific regulatory requirements
- Industry analyses
- Historical precedents
- Economic impacts
- Compliance challenges and opportunities

Conversation History:
$conversation_history$

Output Format Instructions:
$output_format_instructions$"""

    # Set up the Bedrock agent runtime client
    agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')
    
    results = []
    
    # Loop over each knowledge base in the dict
    for kb_id, doc_info in kb_dict.items():
        result = {
            'kb_id': kb_id,
            'response': None,
            'error': None
        }
        
        try:
            # Call the retrieve_and_generate API
            response = agent_runtime_client.retrieve_and_generate(
                input={
                    'text': predefined_prompt
                },
                retrieveAndGenerateConfiguration={
                    'type': 'KNOWLEDGE_BASE',
                    'knowledgeBaseConfiguration': {
                        'knowledgeBaseId': kb_id,
                        'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20240620-v1:0',
                        'generationConfiguration': {
                            'promptTemplate': {
                                'textPromptTemplate': generation_prompt
                            }
                        },
                        'orchestrationConfiguration': {
                            'promptTemplate': {
                                'textPromptTemplate': orchestration_prompt
                            }
                        },
                        'retrievalConfiguration': {
                            'vectorSearchConfiguration': {
                                'numberOfResults': 50
                            }
                        }
                    }
                }
            )
            
            result['response'] = response['output']['text']
            
        except Exception as e:
            result['error'] = str(e)
        
        # format each result for each metric to scale [0; 5]
        r2 = {}
        if result['response']:
            try:
                import json
                response_text = result['response'].strip()
                if response_text.startswith('```json'):
                    response_text = response_text[7:].strip()
                if response_text.endswith('```'):
                    response_text = response_text[:-3].strip()
                analysis = json.loads(response_text)
                
                # Scale stock_price_effect from [-10, 10] to [0, 5]
                stock_price_effect = analysis.get('stock_price_effect', 0)
                scaled_stock_price_effect = (stock_price_effect + 10) / 4  # Scale to [0, 5]
                
                # Scale importance_level from [0, 10] to [0, 5]
                importance_level = analysis.get('importance_level', 0)
                scaled_importance_level = importance_level / 2  # Scale to [0, 5]
                
                r2['score'] = scaled_stock_price_effect
                r2['confiance'] = scaled_importance_level
                r2['nom'] = doc_info['name']
                r2['lien'] = doc_info['link']
                r2['résumé'] = analysis.get('reasoning', '') + " " + analysis.get('impact_on_business', '')
                

            except Exception as e:
                print(f"Error processing KB {kb_id}: {str(e)}")
                result['error'] = f"Error parsing response JSON: {str(e)}"
        
        results.append(r2)
        
    
    return results

# Example usage
# if __name__ == "__main__":
#     # Define the company to analyze
#     company = "Apple"
    
#     # Dictionary with KB IDs and document names
#     kb_dict = {
#         'C17M6LNKIO': {
#             'name': 'DIRECTIVE (UE) 2019/2161 DU PARLEMENT EUROPÉEN ET DU CONSEIL',
#             'link': 'https://eur-lex.europa.eu/legal-content/FR/TXT/?uri=CELEX%3A32019L2161'
#         },
#         'LUZI3KNAXB': {
#             'name': 'H.R.5376 - Inflation Reduction Act of 2022',
#             'link': 'https://www.congress.gov/bill/117th-congress/house-bill/5376/text'
#         },
#         'KDJOOB5LEE': {
#             'name': 'H.R.1 - One Big Beautiful Bill Act',
#             'link': 'https://www.congress.gov/bill/118th-congress/house-bill/1/text'
#         },
#         'QWPHT3J1VL': {
#             'name': '人工知能関連技術の研究開発及び活用の推進に関する法律',
#             'link': 'https://www8.cao.go.jp/cstp/ai/ai_act/ai_act.html'
#         },
#         'B5ZKLSCGZN': {
#             'name': '中华人民共和国能源法__中国政府网',
#             'link': 'https://www.gov.cn/yaowen/liebiao/202411/content_6985761.htm'
#         },
#         '3Y4HFXJI9C': {
#             'name': 'REGULATION (EU) 20241689 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL',
#             'link': 'https://eur-lex.europa.eu/eli/reg/2024/1689/oj/eng'
#         }
#     }
    
#     # Call the function
#     analysis_results = analyze_regulatory_impact(company, kb_dict)
    
#     # Print the results
#     print(analysis_results)

[{'score': 3.0, 'confiance': 4.0, 'nom': 'DIRECTIVE (UE) 2019/2161 DU PARLEMENT EUROPÉEN ET DU CONSEIL', 'lien': 'https://eur-lex.europa.eu/legal-content/FR/TXT/?uri=CELEX%3A32019L2161', 'résumé': "The regulations described focus on enhancing consumer protection, increasing transparency, and imposing stricter penalties for non-compliance in e-commerce and digital services. For a company like Apple, this has several implications: 1) Increased transparency requirements for product rankings and reviews could affect how Apple presents products in its App Store. 2) Stricter rules on personalized pricing and targeted advertising may impact Apple's marketing strategies. 3) The potential for significant fines (up to 4% of annual turnover in EU member states) for violations increases compliance risks. 4) Requirements for clear information on the identity of third-party sellers could affect the App Store ecosystem. 5) Apple's privacy-focused approach may give it a competitive advantage in comply